In [1]:
from requests import get

#get the first page of the east bay housing prices
response = get('https://sfbay.craigslist.org/search/sfc/reb?hasPic=1&search_distance=5&postal=94114&availabilityMode=0&sale_date=all+dates') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')


from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1)

iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

for post in posts:

    if post.find('span', class_ = 'result-hood') is not None:

        #posting date
        #grab the datetime element 0 for date and 1 for time
        post_datetime = post.find('time', class_= 'result-date')['datetime']
        post_timing.append(post_datetime)

        #neighborhoods
        post_hood = post.find('span', class_= 'result-hood').text
        post_hood=re.split(r'[\(\)]',post_hood)
        post_hood=''.join(post_hood)
        post_hoods.append(post_hood)
        

        #title text
        post_title = post.find('a', class_='result-title hdrlnk')
        post_title_text = post_title.text
        post_title_texts.append(post_title_text)

        #post link
        post_link = post_title['href']
        post_links.append(post_link)
            
        #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
        post_price = int(post.a.text.strip().replace("$", "")) 
        post_prices.append(post_price)
            
        if post.find('span', class_ = 'housing') is None:
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)
            sqft=np.nan
            sqfts.append(sqft)
                
        #if the first element is accidentally square footage
        elif 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:

            #make bedroom nan
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)

            #make sqft the first element
            sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
            sqfts.append(sqft)

            #if the length of the housing details element is more than 2
        elif len(post.find('span', class_ = 'housing').text.split()) > 2:

            #therefore element 0 will be bedroom count
            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
            bedroom_counts.append(bedroom_count)

            #and sqft will be number 3, so set these here and append
            sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
            sqfts.append(sqft)

            #if there is num bedrooms but no sqft
        elif len(post.find('span', class_ = 'housing').text.split()) == 2:

            #therefore element 0 will be bedroom count
            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
            bedroom_counts.append(bedroom_count)

            #and sqft will be number 3, so set these here and append
            sqft = np.nan
            sqfts.append(sqft)                    

        else:
            bedroom_count = np.nan
            bedroom_counts.append(bedroom_count)

            sqft = np.nan
            sqfts.append(sqft)

        #if none of those conditions catch, make bedroom nan, this won't be needed    

import pandas as pd

craigslist_options = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices})

In [2]:
Latitude=[]
Longitude=[]
Picture=[]
for i in range(0,len(craigslist_options.iloc[:,2])):
    page_link=get(craigslist_options.iloc[i,5])
    page_link_soup=BeautifulSoup(page_link.text,'html.parser')
    FrontPic=page_link_soup.find('div',class_='slide first visible')
    for link in FrontPic.find_all('img'):
        FrontPic1=link.get('src')
    Picture.append(FrontPic1)
    MapPic=page_link_soup.find('div',id='map')
    MapPicLat=MapPic['data-latitude']
    MapPicLon=MapPic['data-longitude']
    Latitude.append(MapPicLat)
    Longitude.append(MapPicLon)

craigslist_options['Latitude']=Latitude
craigslist_options['Longitude']=Longitude
craigslist_options['Picture Link']=Picture
craigslist_options.head()

,posted,neighborhood,post title,number bedrooms,sqft,URL,price,Latitude,Longitude,Picture Link
0,2020-03-30 22:08,glen park,Great home deals around the city,4,2200.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,1049000,37.758700,-122.433000,https://images.craigslist.org/00g0g_yJSOiN53xK...
1,2020-04-01 13:44,castro / upper market,Great Opporunity | 680 Sanchez Street | 3/3,3,2128.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2695000,37.758255,-122.430297,https://images.craigslist.org/01212_aV1UCFNRN0...
2,2020-04-02 18:29,castro / upper market,Amazing Development Opportunity | Corona Heights,NaN,NaN,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2195000,37.764354,-122.440962,https://images.craigslist.org/00H0H_akle652g3W...
3,2020-04-06 15:50,mission district,Mixed use building on corner lot. Reduced price!,NaN,7500.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2300000,37.763557,-122.418787,https://images.craigslist.org/00G0G_1cLkSBnbzA...
4,2020-04-06 15:51,mission district,Well maintained office building for sale!,NaN,30000.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,13500000,37.764057,-122.418680,https://images.craigslist.org/00B0B_agTyaKYbYk...


In [4]:
import pickle
craigslist_options.to_pickle('craigslistdf.sav')